In [1]:
import geopandas as gpd 
from rasterstats import zonal_stats
import pandas as pd
import numpy as np
import rasterio
from rasterio import features

In [2]:
vec = gpd.read_file('../../disturbances_planted/disturbances_planted.gdb')

geom = [(geometry, value) for geometry, value in zip(vec.geometry, vec.ACTIVITY_TREATMENT_UNIT_ID)]

with rasterio.open(r'C:\dev\GIS-270\weekly_max_ndvi\data\clipped\agextent.mod.bestquality.maxndvi.2003076.2003082.tif') as ref: 
    meta = ref.meta.copy()
    height, width = ref.shape
    transform = ref.transform
    print(meta)

rasterized = features.rasterize(geom, 
                                out_shape = (height, width),
                                fill=0,
                                transform=transform,
                                all_touched=True,
                                default_value=0,
                                dtype=None)

rasterized

    

c:\dev\GIS-270\.venv\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_read(


{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.4028230607370965e+38, 'width': 5475, 'height': 4685, 'count': 1, 'crs': CRS.from_wkt('PROJCS["NAD83 / BC Albers",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",45],PARAMETER["longitude_of_center",-126],PARAMETER["standard_parallel_1",50],PARAMETER["standard_parallel_2",58.5],PARAMETER["false_easting",1000000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","3005"]]'), 'transform': Affine(291.7322303406986, 0.0, 273585.0522421824,
       0.0, -291.7322303406986, 1735803.3735694718)}


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [65]:
with rasterio.open('test_raster.tif', 'w', **meta) as dst: 
    dst.write(rasterized, 1)

In [3]:
rasterized.shape



(4685, 5475)

In [32]:
len(vec.ACTIVITY_TREATMENT_UNIT_ID.unique())

15567

In [22]:
def zonal_stats_array(data, zones, nodata_val):
    print(data, zones)

    
    # First, group all missing data into one zone for easy handling
    zones = np.where(data != nodata_val, zones, nodata_val)
    
    # Create pandas dataframe
    df = pd.DataFrame({'ACTIVITY_TREATMENT_UNIT_ID': zones, 'Value': data})
    
    # Remove no data zones 
    df = df[df.ACTIVITY_TREATMENT_UNIT_ID != nodata_val] 
    
    # Calculate stats and add results to table as a column
    df = df.groupby('ACTIVITY_TREATMENT_UNIT_ID').agg(
         {'Value': ['min', 'max', 'mean', 'median', 'std', 'count']}
         )
    
    # Drop multilevel column and rename columns if necessary
    df = df.droplevel(0, axis=1).rename(
         columns = {'min': 'MinValue',
                    'max': 'MaxValue',
                    'mean': 'MeanValue',
                    'median': 'MedianValue',
                    'std': 'StdevValue',
                    'count': 'CountValue'})
    
    # Removes ID column from index and makes a column again
    df.reset_index(inplace = True)
    
    return(df)

In [23]:
with rasterio.open('../../weekly_max_ndvi/data/clipped/agextent.mod.bestquality.maxndvi.2003076.2003082.tif') as src:
    zs = zonal_stats_array(src.read(1).ravel(), rasterized.ravel(), src.nodata)

[-3.402823e+38 -3.402823e+38 -3.402823e+38 ... -3.402823e+38 -3.402823e+38
 -3.402823e+38] [0 0 0 ... 0 0 0]


In [40]:
import datetime


tiff = 'agextent.mod.bestquality.maxndvi.2003076.2003082.tif'
tiff_split = tiff.split('.')
date = tiff_split[4]


zs['date'] = datetime.datetime.strptime(date, "%Y%j")
zs 

,ACTIVITY_TREATMENT_UNIT_ID,MinValue,MaxValue,MeanValue,MedianValue,StdevValue,CountValue,date
0,0.0,0.000000,1.000000,0.499189,0.508946,0.183635,823638,2003-03-17
1,538478.0,0.391779,0.756124,0.560025,0.532173,0.183763,3,2003-03-17
2,1640150.0,0.358072,0.358072,0.358072,0.358072,NaN,1,2003-03-17
3,1640185.0,0.333333,0.333333,0.333333,0.333333,NaN,1,2003-03-17
4,1640192.0,0.653801,0.653801,0.653801,0.653801,NaN,1,2003-03-17
...,...,...,...,...,...,...,...,...
5554,4123243.0,0.246267,0.456442,0.362924,0.374494,0.089896,4,2003-03-17
5555,4123414.0,0.215226,0.338200,0.278478,0.282008,0.061563,3,2003-03-17
5556,4123456.0,0.373619,0.412568,0.399585,0.412568,0.022487,3,2003-03-17
5557,4123657.0,0.270826,0.523749,0.432062,0.449701,0.088508,9,2003-03-17


In [7]:
len(np.unique(rasterized))

14333

In [11]:
treatment = gpd.read_file('../../disturbances_planted/disturbances_planted.gdb').sort_values('ACTIVITY_TREATMENT_UNIT_ID')
len(treatment.ACTIVITY_TREATMENT_UNIT_ID.unique())

15567

In [15]:
unique = np.unique(rasterized)
not_in = [x for x in treatment.ACTIVITY_TREATMENT_UNIT_ID if x not in unique]

In [16]:
print(not_in)

[1610937, 1610955, 1610959, 1610967, 1610999, 1611027, 1611032, 1640151, 1640156, 1640158, 1640162, 1640164, 1640171, 1640173, 1640174, 1640175, 1640176, 1640179, 1640180, 1640181, 1640200, 1640205, 1640206, 1640215, 1640230, 1640231, 1640240, 1640241, 1640242, 1640247, 1640248, 1695416, 1695436, 1695437, 1695440, 1695442, 1695447, 1695451, 1695464, 1695465, 1695468, 1695469, 1695492, 1695499, 1695508, 1695509, 1730693, 1747781, 1747784, 1747785, 1747810, 1747811, 1747814, 1747955, 1747956, 1747957, 1747968, 1747972, 1747981, 1748477, 1748489, 1748490, 1748497, 1748506, 1748534, 1748546, 1748572, 1748576, 1748585, 1748608, 1748609, 1748614, 1748631, 1748658, 1748659, 1748880, 1748883, 1779923, 1779924, 1821867, 1821881, 1821885, 1821892, 1821894, 1821903, 1821915, 1822022, 1822033, 1822034, 1822042, 1822077, 1822093, 1822094, 1822097, 1822104, 1822110, 1822116, 1822118, 1822121, 1822141, 1822161, 1822162, 1822167, 1822170, 1822215, 1905334, 1967202, 2051549, 2051562, 2051571, 2051602, 

In [17]:
len(not_in)

1235

In [26]:
zones_df = pd.DataFrame({'ACTIVITY_TREATMENT_UNIT_ID': np.unique(rasterized.ravel())})

In [24]:
zs

,ACTIVITY_TREATMENT_UNIT_ID,MinValue,MaxValue,MeanValue,MedianValue,StdevValue,CountValue
0,0.0,0.000000,1.000000,0.499189,0.508946,0.183635,823638
1,538478.0,0.391779,0.756124,0.560025,0.532173,0.183763,3
2,1640150.0,0.358072,0.358072,0.358072,0.358072,NaN,1
3,1640185.0,0.333333,0.333333,0.333333,0.333333,NaN,1
4,1640192.0,0.653801,0.653801,0.653801,0.653801,NaN,1
...,...,...,...,...,...,...,...
5554,4123243.0,0.246267,0.456442,0.362924,0.374494,0.089896,4
5555,4123414.0,0.215226,0.338200,0.278478,0.282008,0.061563,3
5556,4123456.0,0.373619,0.412568,0.399585,0.412568,0.022487,3
5557,4123657.0,0.270826,0.523749,0.432062,0.449701,0.088508,9


In [30]:
joined = zones_df.set_index('ACTIVITY_TREATMENT_UNIT_ID').join(zs.set_index('ACTIVITY_TREATMENT_UNIT_ID'), on='ACTIVITY_TREATMENT_UNIT_ID')

In [42]:
joined.reset_index().count()

ACTIVITY_TREATMENT_UNIT_ID    14333
MinValue                       5559
MaxValue                       5559
MeanValue                      5559
MedianValue                    5559
StdevValue                     4592
CountValue                     5559
dtype: int64